# <div align="center" style="color: #ff5733;">TSA Account Closure Starting File</div>

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

# *Check if the Data in the table is EOD yesterday*

In [2]:
sq = """select Run_Date, count(1)cnt from risk_credit_mis.tsa_account_closure group by 1 order by 1 desc"""
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')
df.head(1)

Job ID 76d60a13-07b1-4793-9679-da3a6ce53cf8 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,Run_Date,cnt
0,2024-11-25,714021


# Some Date Engineering

In [3]:
d = '2024-11-25'   ### Need to change this to D-1 everytime we run this code ###
# Get the current date
current_date = pd.Timestamp.now().date()
d_minus_1_days = current_date - pd.DateOffset(days=1)
# Calculate D-60 days
d_minus_60_days = d_minus_1_days - pd.DateOffset(days=60)
# Calculate D-180 days
d_minus_180_days = d_minus_1_days - pd.DateOffset(days=180)

print("D-1 day from current date", d_minus_1_days )
print("D-60 day from the current date - 1:", d_minus_60_days)
print("D-180 day from the current date -1:", d_minus_180_days)

d_minus_1_days_string = d_minus_1_days.strftime('%Y-%m-%d')
print("D-1 day from the current date:", d_minus_1_days_string)

d_minus_60_days_string = d_minus_60_days.strftime('%Y-%m-%d')
print("D-60 day from the current date:", d_minus_60_days_string)

d_minus_180_days_string = d_minus_180_days.strftime('%Y-%m-%d')
print("D-180 day from the current date:", d_minus_180_days_string)

D-1 day from current date 2024-11-25 00:00:00
D-60 day from the current date - 1: 2024-09-26 00:00:00
D-180 day from the current date -1: 2024-05-29 00:00:00
D-1 day from the current date: 2024-11-25
D-60 day from the current date: 2024-09-26
D-180 day from the current date: 2024-05-29


Step1: At line 11 change the date to D-1<br><br>
Step2: At line around 36 (and (TSA_Opening_date < date '2023-10-18')   ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) Change the date D-60 Days <br><br>
Step3: At line around 45 (WHEN tsa_status = 'Active' and Last_TSA_EOD_Balance_Date is null and (TSA_Opening_date < date '2023-10-18')  ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) Change the date D-60 Days <br><br>
Step4: At line 58 (and TSA_Opening_date < date '2023-06-18'  and (Last_TSA_Credit_Date < date '2023-06-18' and Last_TSA_Debit_Date < date '2023-06-18' and coalesce(Last_Group_Stash_Credit_Date,date '1970-01-01') < date '2023-06-18')  ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) <br><br>
Step5: At line 72 (Run_Date = date '2024-01-09'###DATE change here to D-1) Change the date to D-1<br><br>

In [4]:
sq = f"""with
w_kyc
as
(
  select distinct PARTYID, SEGCATEGORYDESC, row_number() over(partition by partyid order by SEGEFFECTIVEFROM desc) as rnk from `core_raw.party_details`
)
,
w_tsa
as
(
  select * from risk_credit_mis.tsa_account_closure where Run_Date = date '{d_minus_1_days_string}'  ###DATE change here to D-1
  and
  Customer_Id not in (select cast(PARTYID as string) from w_kyc where rnk = 1 and SEGCATEGORYDESC = 'BKYC')
)
,
w_approved
as
(
  select DISTINCT customerId from risk_credit_mis.loan_master_table where loantype = 'BIGLOAN' OR FLAGAPPROVAL = 1 OR FLAGDISBURSEMENT = 1  
  or (stage1Status = 'EXPIRED' AND cddInitiatedDateAndTime IS NULL)
)
select *
from
(
  select 
  a.Customer_Id,a.TSA_Opening_date,
  c.OFSTANDARDACCOUNTID as TSA_Account_Number, c.OfblockAmount, 
  a.TSA_Status,a.TSA_Balance,a.Solo_Stash_Balance,a.Group_Stash_Balance,a.Term_Deposit_Balance,a.Active_Term_Deposit_flag,a.Last_TSA_Credit_Date,a.Last_TSA_Debit_Date,
  a.ARI_Flag,a.Active_Loan,a.Total_Loan_Outstanding,  case when b.customerId is not null then 1 else 0 end as Approved_Loans,
  a.Pending_Loan_Applications,a.Hard_Reject_Loans_App,a.Soft_Reject_Loans_App,a.Cancelled_Loans_App, extract(date from a.Last_Login_Date) as Last_Login_Date,
  a.Last_TSA_EOD_Balance_Date,a.Last_Group_Stash_Credit_Date,a.Group_Stash_Balance_Owner,a.Run_Date,a.Batch,a.Processing_Month,a.pending_loans_not_expired_not_exempt,a.expired_loans,
  a.pending_loans_lastDt,a.expired_loans_lastDt,
  CASE 
    WHEN tsa_status = 'Active' and (Last_TSA_Credit_Date = date '1970-01-01' and Last_Group_Stash_Credit_Date is null)
    and (TSA_Opening_date < date '{d_minus_60_days_string}')                                                    ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
    and ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) = 0 )
    and (Active_Loan = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
    and (Active_Term_Deposit_flag = 0)
    and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)  THEN 'TYPE 1'
    and b.customerId is null  THEN 'TYPE 1'


    WHEN tsa_status = 'Active' and Last_TSA_EOD_Balance_Date is null 
    and (TSA_Opening_date < date '{d_minus_60_days_string}')                                                    ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
    and ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) = 0 )
    and (Active_Loan = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
    and (Active_Term_Deposit_flag = 0)
    and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)
    and b.customerId is null  
    and (date_trunc(TSA_Opening_date,day) = date_trunc(Last_Login_Date,day)) THEN 'TYPE 2'

    
    WHEN ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) < 1 )
    and (Active_Loan = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
    and (Active_Term_Deposit_flag = 0)
    and TSA_Opening_date < date '{d_minus_180_days_string}'                                                    ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
    and (Last_TSA_Credit_Date < date '{d_minus_180_days_string}'                                                    ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
    and Last_TSA_Debit_Date < date '{d_minus_180_days_string}'                                                 ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
    and coalesce(Last_Group_Stash_Credit_Date,date '1970-01-01') < date '{d_minus_180_days_string}')           ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
    and tsa_status = 'Active'
    and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)
    and b.customerId is null  
    THEN 'TYPE 3'
    ELSE 'NO ACTION'
    END AS ACTION_TYPE
  from w_tsa A
  left join w_approved b
  on cast(a.Customer_Id as int64) = b.customerId
  left join core_raw.customer_accounts c
  on c.OFCUSTOMERID = a.Customer_Id and c.CRINTERDESC = 'Transactional Savings Account Inv_R' and c.OFISCLOSED = 'N'
  where
  Run_Date = date '{d_minus_1_days_string}'                                                                         ###DATE change here to D-1
)
"""
print(sq)

with
w_kyc
as
(
  select distinct PARTYID, SEGCATEGORYDESC, row_number() over(partition by partyid order by SEGEFFECTIVEFROM desc) as rnk from `core_raw.party_details`
)
,
w_tsa
as
(
  select * from risk_credit_mis.tsa_account_closure where Run_Date = date '2024-11-25'  ###DATE change here to D-1
  and
  Customer_Id not in (select cast(PARTYID as string) from w_kyc where rnk = 1 and SEGCATEGORYDESC = 'BKYC')
)
,
w_approved
as
(
  select DISTINCT customerId from risk_credit_mis.loan_master_table where loantype = 'BIGLOAN' OR FLAGAPPROVAL = 1 OR FLAGDISBURSEMENT = 1  
  or (stage1Status = 'EXPIRED' AND cddInitiatedDateAndTime IS NULL)
)
select *
from
(
  select 
  a.Customer_Id,a.TSA_Opening_date,
  c.OFSTANDARDACCOUNTID as TSA_Account_Number, c.OfblockAmount, 
  a.TSA_Status,a.TSA_Balance,a.Solo_Stash_Balance,a.Group_Stash_Balance,a.Term_Deposit_Balance,a.Active_Term_Deposit_flag,a.Last_TSA_Credit_Date,a.Last_TSA_Debit_Date,
  a.ARI_Flag,a.Active_Loan,a.Total_Loan_Outstanding,  case whe

In [5]:
df = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(f"The count of rows and columns in the extracted data are\t:{df.shape}")

Job ID 637181b6-a26c-494a-9789-ceac4a91dc28 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The count of rows and columns in the extracted data are	:(680466, 32)


In [7]:
df.shape

(680466, 32)

In [8]:
df.columns

Index(['Customer_Id', 'TSA_Opening_date', 'TSA_Account_Number',
       'OfblockAmount', 'TSA_Status', 'TSA_Balance', 'Solo_Stash_Balance',
       'Group_Stash_Balance', 'Term_Deposit_Balance',
       'Active_Term_Deposit_flag', 'Last_TSA_Credit_Date',
       'Last_TSA_Debit_Date', 'ARI_Flag', 'Active_Loan',
       'Total_Loan_Outstanding', 'Approved_Loans', 'Pending_Loan_Applications',
       'Hard_Reject_Loans_App', 'Soft_Reject_Loans_App', 'Cancelled_Loans_App',
       'Last_Login_Date', 'Last_TSA_EOD_Balance_Date',
       'Last_Group_Stash_Credit_Date', 'Group_Stash_Balance_Owner', 'Run_Date',
       'Batch', 'Processing_Month', 'pending_loans_not_expired_not_exempt',
       'expired_loans', 'pending_loans_lastDt', 'expired_loans_lastDt',
       'ACTION_TYPE'],
      dtype='object')

In [9]:
df.ACTION_TYPE.value_counts(dropna= False)

ACTION_TYPE
NO ACTION    618558
TYPE 1        60115
TYPE 3         1729
TYPE 2           64
Name: count, dtype: int64

In [10]:
df.to_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Marvin\TSA Account Closure\November_December2024\FirstNotification\202411126_TSA_First_Notificaiton.xlsx", index = False, sheet_name='TSA_First_Notification')

In [11]:
# Unique customer count by Action Type
df.groupby('ACTION_TYPE')['Customer_Id'].nunique()

ACTION_TYPE
NO ACTION    618546
TYPE 1        60115
TYPE 2           64
TYPE 3         1729
Name: Customer_Id, dtype: int64

In [12]:
# Unique Customer Count by Action type when Action type is not like No Action
df[df['ACTION_TYPE']!= 'NO ACTION'].groupby('ACTION_TYPE')['Customer_Id'].nunique()

ACTION_TYPE
TYPE 1    60115
TYPE 2       64
TYPE 3     1729
Name: Customer_Id, dtype: int64

In [13]:
dfd = df[df['ACTION_TYPE'] != 'NO ACTION'].groupby('ACTION_TYPE').agg(
    Customer_Unique_Count=('Customer_Id', 'nunique')
).reset_index()

dfd

,ACTION_TYPE,Customer_Unique_Count
0,TYPE 1,60115
1,TYPE 2,64
2,TYPE 3,1729
